# Step 1:  Import Libraries, Prepare Robot and Set-Up Camera

- Xarm SDK reference: https://github.com/xArm-Developer/xArm-Python-SDK/tree/master

In [1]:
# import libraries
import pyrealsense2 as rs
import numpy as np
import cv2
import os
import pandas as pd
import time
from xarm.wrapper import XArmAPI
import cv2.aruco as aruco
import numpy as np
from scipy.spatial.transform import Rotation as R

d:\Anaconda New Install\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


SDK_VERSION: 1.14.8


In [63]:
# set-up camera
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.rgb8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

In [64]:
#prepare robot
arm = XArmAPI('192.168.1.117')  # Replace with your robot's IP address
arm.connect()
arm.clean_warn()
arm.clean_error()
arm.motion_enable(enable=True)
arm.set_mode(0)      # Position mode
arm.set_state(0)     # Ready state

ROBOT_IP: 192.168.1.117, VERSION: v2.5.5, PROTOCOL: V1, DETAIL: 6,12,FX8501,AC8500,v2.5.5, TYPE1300: [1, 1]
change protocol identifier to 3
[motion_enable], xArm is not ready to move
[set_state], xArm is ready to move


0

In [65]:
# Beginning pose of robot (easy for mounting checkerboard)
pose = [-50.810581, -450.742157, 210.200928, -90, 0, -90]
arm.set_position(x = pose[0], y=pose[1], z=pose[2], roll=pose[3], pitch=pose[4], yaw=pose[5], speed=50)

0

# Step 2: Attach Checkerboard to Robot

- You can choose between a normal checkerboard and the ChArUco version, however we do stimulate to take the second option. In that case the camera can be moved closer to the scene as the ChArUco allows for only partial display which keeps the variety in poses high.
- Checkerboards can be printed here: https://calib.io/pages/camera-calibration-pattern-generator?srsltid=AfmBOorLrMXPaB0gHMyK0HNOnLuCq6Y1MaLlo5pN0_-Bl8UeyUumNPad


# Step 3: Define Checkerboard Pattern and Get Camera Intrinsics

In [66]:
print(dir(cv2.aruco))
import cv2
print(cv2.__version__)

['ARUCO_CCW_CENTER', 'ARUCO_CW_TOP_LEFT_CORNER', 'ArucoDetector', 'Board', 'CORNER_REFINE_APRILTAG', 'CORNER_REFINE_CONTOUR', 'CORNER_REFINE_NONE', 'CORNER_REFINE_SUBPIX', 'CharucoBoard', 'CharucoDetector', 'CharucoParameters', 'DICT_4X4_100', 'DICT_4X4_1000', 'DICT_4X4_250', 'DICT_4X4_50', 'DICT_5X5_100', 'DICT_5X5_1000', 'DICT_5X5_250', 'DICT_5X5_50', 'DICT_6X6_100', 'DICT_6X6_1000', 'DICT_6X6_250', 'DICT_6X6_50', 'DICT_7X7_100', 'DICT_7X7_1000', 'DICT_7X7_250', 'DICT_7X7_50', 'DICT_APRILTAG_16H5', 'DICT_APRILTAG_16h5', 'DICT_APRILTAG_25H9', 'DICT_APRILTAG_25h9', 'DICT_APRILTAG_36H10', 'DICT_APRILTAG_36H11', 'DICT_APRILTAG_36h10', 'DICT_APRILTAG_36h11', 'DICT_ARUCO_MIP_36H12', 'DICT_ARUCO_MIP_36h12', 'DICT_ARUCO_ORIGINAL', 'DetectorParameters', 'Dictionary', 'Dictionary_getBitsFromByteList', 'Dictionary_getByteListFromBits', 'EstimateParameters', 'GridBoard', 'RefineParameters', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_native', 'calibr

In [67]:
square_size = 0.025
marker_size = 0.018
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

charuco_board = aruco.CharucoBoard(
    (7,5),                # size as a tuple (squaresX, squaresY)
    square_size,
    marker_size,
    dictionary
)

In [68]:
profile = pipeline.start(config)

intr = profile.get_stream(rs.stream.color).as_video_stream_profile().get_intrinsics()

camera_matrix = np.array([
    [intr.fx,    0,      intr.ppx],
    [0,       intr.fy,   intr.ppy],
    [0,          0,      1]
])

print(camera_matrix)

pipeline.stop()

[[608.46307373   0.         309.3677063 ]
 [  0.         606.87322998 213.83302307]
 [  0.           0.           1.        ]]


# Step 4: Predefine Functions
- Reference for calibration and part of code beneath from OpenCV forum: https://forum.opencv.org/t/eye-to-hand-calibration/5690

In [69]:
def get_charuco_object_points(charuco_ids, board):
    squares_x, squares_y = board.getChessboardSize()
    square_length = board.getSquareLength()
    
    object_points = []
    for idx in charuco_ids.flatten():
        x = idx % (squares_x - 1)
        y = idx // (squares_x - 1)
        object_points.append([x * square_length, y * square_length, 0.0])
        
    return np.array(object_points, dtype=np.float32)

In [70]:
def get_image(show_image=True):

    for i in range(30):  # warm-up
        pipeline.wait_for_frames()
        
    align_to = rs.stream.color
    align = rs.align(align_to)

    # Main loop
    frames = pipeline.wait_for_frames()
    aligned_frames = align.process(frames)

    color_frame = aligned_frames.get_color_frame()
    color_image = np.asanyarray(color_frame.get_data())

    # Save and convert to grayscale
    cv2.imwrite('checkerboard_image.png', color_image)

    gray = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)  # Correct now

    if show_image:
        cv2.imshow("Checkerboard", gray)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    return gray

In [71]:
def get_matrix_targetcam_charuco(gray, charuco_board, camera_matrix):
    # 1. Detect markers
    dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
    parameters = aruco.DetectorParameters()
    corners, ids, rejected = aruco.detectMarkers(gray, dictionary, parameters=parameters)
    dist_coeffs = np.zeros((5, 1))
    
    if ids is None or len(ids) == 0:
        print("No markers detected")
        return None, None, None

    # 2. Interpolate Charuco corners
    retval, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(corners, ids, gray, charuco_board)
    if retval < 4:  # Need at least 4 corners to solve PnP
        print("Not enough Charuco corners detected")
        return None, None, None

    # 3. Solve PnP with Charuco corners
    object_points = get_charuco_object_points(charuco_ids, charuco_board)
    success, rvec, tvec = cv2.solvePnP(object_points, charuco_corners, camera_matrix, dist_coeffs)
    if not success:
        print("solvePnP failed")
        return None, None, None

    # 4. Convert rotation vector to matrix
    R_target2cam, _ = cv2.Rodrigues(rvec)

    # 5. Build homogeneous transformation matrix
    H_target2cam = np.eye(4)
    H_target2cam[:3, :3] = R_target2cam
    H_target2cam[:3, 3] = tvec.squeeze()

    return H_target2cam, R_target2cam, tvec

In [72]:
def euler_to_matrix(rx, ry, rz, degrees=True):
    """
    Convert Euler angles to a 3x3 rotation matrix.
    Assumes XYZ order: roll (X), pitch (Y), yaw (Z).
    
    Set degrees=True if input is in degrees.
    """
    r = R.from_euler('xyz', [rx, ry, rz], degrees=degrees)
    return r.as_matrix()

In [73]:
# Define pose limits
x_range = [-320, 120]     # forward/back
y_range = [-640, -280]  # left/right
z_range = [180,290]     # height above table
rx_range = [-80, -100]     # roll (degrees)
ry_range = [-30, +30]     # pitch
rz_range = [-60, -110]   # yaw

In [74]:
def generate_random_pose():
    x = np.random.uniform(*x_range)
    y = np.random.uniform(*y_range)
    z = np.random.uniform(*z_range)
    rx = np.random.uniform(*rx_range)
    ry = np.random.uniform(*ry_range)
    rz = np.random.uniform(*rz_range)
    return [x, y, z, rx, ry, rz]

In [75]:
R_gripper2base_list = []
t_gripper2base_list = []
R_target2cam_list   = []
t_target2cam_list   = []

# Step 5: Collect between 20 and 40 calibration poses
- The most important is to get a high variety of poses, set the camera as such that it is not too far from scene but still can see the checkerboard pattern.
- In case of error during calibration you can just go rerun untill you reach your desired amount of poses
- Set time.sleep to at least 6 to allow pose variations to take place

In [80]:
pipeline.start(config)
for i in range(40):
    pose = generate_random_pose()
    arm.set_position(x=pose[0], y=pose[1], z=pose[2], roll=pose[3], pitch=pose[4], yaw=pose[5], speed=60)
    time.sleep(7)
    
    gray = get_image()
    
    # Use Charuco pose estimation
    H1, R1, t1 = get_matrix_targetcam_charuco(gray, charuco_board, camera_matrix)
    if H1 is None:
        print(f"Pose estimation failed at iteration {i}")
        continue
    else:
        R_target2cam_list.append(R1)
        t_target2cam_list.append(t1)

        x, y, z, rx, ry, rz = arm.get_position()[1]
        R2 = euler_to_matrix(rx, ry, rz)
        t2 = np.array([x, y, z]) / 1000  # assuming arm units are mm, convert to meters

        R_gripper2base_list.append(R2)
        t_gripper2base_list.append(t2)

Not enough Charuco corners detected
Pose estimation failed at iteration 1
Not enough Charuco corners detected
Pose estimation failed at iteration 6


KeyboardInterrupt: 

In [81]:
pipeline.stop()

In [82]:
print(len(t_target2cam_list))

15


# Step 6: Calculate final matrix
- - Reference for calibration and part of code beneath from OpenCV forum: https://forum.opencv.org/t/eye-to-hand-calibration/5690

In [83]:
def calibrate_eye_hand(R_gripper2base, t_gripper2base, R_target2cam, t_target2cam, eye_to_hand=True):

    if eye_to_hand:
        # change coordinates from gripper2base to base2gripper
        R_base2gripper, t_base2gripper = [], []
        for Rs, t in zip(R_gripper2base, t_gripper2base):
            R_b2g = Rs.T
            t_b2g = -R_b2g @ t
            R_base2gripper.append(R_b2g)
            t_base2gripper.append(t_b2g)
        
        # change parameters values
        R_gripper2base = R_base2gripper
        t_gripper2base = t_base2gripper

    # calibrate
    Rs, t = cv2.calibrateHandEye(
        R_gripper2base=R_gripper2base,
        t_gripper2base=t_gripper2base,
        R_target2cam=R_target2cam,
        t_target2cam=t_target2cam,
    )

    return Rs, t

In [84]:
R_cam2gripper = np.zeros((3,3))
t_cam2gripper = np.zeros((3,1))

R_cam2gripper, t_cam2gripper = calibrate_eye_hand(
    R_gripper2base_list,
    t_gripper2base_list,
    R_target2cam_list,
    t_target2cam_list
)

In [ ]:
H_cam2gripper = np.eye(4)
H_cam2gripper[:3, :3] = R_cam2gripper
H_cam2gripper[:3, 3] = t_cam2gripper.squeeze()

print(H_cam2gripper)

H_cam2base = H_cam2gripper
H_cam2base[:3,3] *= 1000
print(H_cam2base)
np.savetxt('cam2base.txt', H_cam2base)

[[ 1.45201165e-04  4.82279046e-01 -8.76017637e-01  4.70482633e-01]
 [ 9.90251720e-01  1.21950547e-01  6.73022741e-02 -5.53166582e-01]
 [ 1.39289307e-01 -8.67487744e-01 -4.77559947e-01  3.56808142e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 1.45201165e-04  4.82279046e-01 -8.76017637e-01  4.70482633e+02]
 [ 9.90251720e-01  1.21950547e-01  6.73022741e-02 -5.53166582e+02]
 [ 1.39289307e-01 -8.67487744e-01 -4.77559947e-01  3.56808142e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


[SDK][ERROR][2025-08-07 10:59:20][base.py:247] - - [report-socket] recv error: [WinError 10054] De externe host heeft een verbinding verbroken
[SDK][ERROR][2025-08-07 10:59:20][base.py:293] - - [main-socket] recv error: [WinError 10054] De externe host heeft een verbinding verbroken
